# Hebbian Rule - Thymio Learns

In [1]:
# If you did not install tdmclient, try "!pip install tdmclient", 
# or have a look at the tutorial "Control your Thymio in Python"
import tdmclient.notebook
await tdmclient.notebook.start()



In [88]:
%%run_python

w_l = [0,0,0,0,0,0,0, 0]  #8
w_r = [0,0,0,0,0,0,0, 0]  #8 
random_sign = 0
tmp = 0
tt = 0 

def random():
    global w_l, w_r
    for i in range(8):
        random_value = -20 + (math_rand() % 41)
        w_l[i] = random_value
        tmp = 20 + (math_rand() % 41)
        tmp = tmp%2
        if tmp == 0:
            random_sign = 1 
        else:
            random_sign = -1 
            
        w_r[i] = random_value * random_sign
        
    
random()

In [2]:
%%run_python
#%%transpile_to_aseba # Uncomment to show the code generated in Aseba

# Initially, the robot does not move
# To train the robot to move forwards when an object is not
#   detected, touch the center button
# When the center / rear / left / right sensor detects your
#   finger, touch the back / front / right /left button
# Do the training one or more times
# The robot will update the ANN weights
#   and avoid obstacles

w_l = [0,0,0,0,0,0,0, 0]  #8
w_r = [0,0,0,0,0,0,0, 0]  #8 

#w_l = [40,  -50, -30, -20, -40,  -30, -10, 10]
#w_r = [-40, -50, -30,  20,  40, -10, 30, 10]

x = [0,0,0,0,0,0,0, 0]
y = [0,0]

# Scale factors for sensors, outputs and motors
SENSOR_SCALE = 100
OUTPUT_SCALE  = 20
MOTOR_SCALE  = 15
    
# Learning rate
ALPHA = 1
    
# Speed increment for each learning episode
SPEED = 10

# The desired output for each button: +/-speed
y_left = 0
y_right = 0

#support variable initialization                          
training_state = False    # if training_state = ff -> Training phase  else -> Test phase
action_performed = False
feedback_received = False
nf_leds_top(0,32,0)

# Time period (millisecconds) to compute motor outputs from inputs
timer_period[0] = 100
timer_period[1] = 500

tap_counter = 0
double_tap_detected = False
counter = 0

timer_c = 0
moving = False

leds_top = [0,0,0]
leds_circle = [0,0,0,0,0,0,0, 0]

motor_left_target  = 0
motor_right_target = 0


def initialize_random_weights():
    #inizialize w_l randomly with values between [-60, 60] 
    #w_r is a copy of w_l but the +/- sign is assigned randomly  
    #we cannot proceed with a complete random generation of both array because value could diverge over utilization
    global w_l, w_r
    for i in range(8):
        random_value = -20 + (math_rand() % 41)
        #random sign generation 
        tmp = 20 + (math_rand() % 41)
        tmp = tmp%2
        if tmp == 0:
            random_sign = 1 
        else:
            random_sign = -1 
            
        w_l[i] = random_value
        w_r[i] = random_value * random_sign
     

initialize_random_weights() 


def weights_and_lights():
    global w_l, w_r, leds_circle
    intensity_weights = [0,0,0,0,0,0,0, 0]
    order = [6,7,0,1,2,3,5,4]
    max_val = w_l[0]
    min_val = w_l[0]
    max_intensity = 32
    min_intensity = 0
    
    # Iterate through the array to find the maximum value
    for i in range(8):
        if w_l[i] > max_val:
            max_val = w_l[i]

    # Iterate through the array to find the minimum value -> could be replaced with function math_max() of aseba 
    for i in range(8):
        if w_l[i] < min_val:
            min_val = w_l[i]
        
    #mapping weight to led intensity
    for i in range(8):
        if w_l[i] == 0: #turn off the sensor 
            intensity_weights[order[i]] = 0
        else: 
            intensity_weights[order[i]] = ((w_l[i] - min_val) * (max_intensity - min_intensity)) // (max_val - min_val) + min_intensity
            
    leds_circle = intensity_weights

def change_weights():
    global w_l, w_r, ALPHA, y_left, y_right, x, OUTPUT_SCALE, feedback_received 
    for i in range(8):
        w_l[i]  = w_l[i]  + (ALPHA*y_left*x[i])  // OUTPUT_SCALE
        w_r[i] = w_r[i] + (ALPHA*y_right*x[i]) // OUTPUT_SCALE    

    weights_and_lights()
    feedback_received = True

def reset_weights(): 
    global w_l, w_r, y_left, y_right, training_state, action_performed, feedback_received, moving 
    y_left  = 0 
    y_right = 0 
    training_state    = False
    action_performed  = False
    feedback_received = False
    moving = False
    nf_leds_circle(0,0,0,0,0,0,0,0)
    
    for i in range(8):
        w_l[i]  = 0
        w_r[i] = 0
        

def long_press_detection(button):
    global counter 
    if button == 1: 
        timer_pressed = counter
    if button == 0: 
        timer_release = counter
    timer_diff = timer_release - timer_pressed
   
    return timer_diff > 2


def perceive_and_act(sensor_scale, motor_scale):
    global prox_horizontal, motor_left_target, motor_right_target, x, y, w_l, w_r 
    no_obstacle = True

    for i in range(7):
        # Get and scale sensor input if something is detected
        x[i] = prox_horizontal[i] // sensor_scale
        
        if x[i] != 0:  # If any sensor detects an obstacle, set no_obstacle to False
            no_obstacle = False

    if no_obstacle:  #set x[8] to 100 if no obstacles are detected 0 otherwise (default)
        x[7] = 100 
    else: 
        x[7] = 0 
        
    # Compute dot product of inputs and weights
    y = [0,0]    
        
    for i in range(8):    
        y[0] = y[0] + x[i] * w_l[i]
        y[1] = y[1] + x[i] * w_r[i]
        
    motor_left_target = y[0] // motor_scale
    motor_right_target = y[1] // motor_scale     #True because action has been performed 
    
    return True



    
# For each button set y_left, y_right and change weights, center erase them
@onevent
def button_center():
    global w_l, w_r, training_state, tap_counter, double_tap_detected
    tap_counter = (tap_counter + 1) 
    
    if tap_counter > 3:
       double_tap_detected = True
       nf_leds_top(32,0 ,0)
       reset_weights()
    elif button_center == 1:       #change state 
        if training_state == False: 
            #define led color for training and test state
            training_state = True
            action_performed = False
            nf_leds_top(0,0,32)   #Arguments are in the order red, green, blue
        else:
            training_state = False
            nf_leds_top(0,32,0)   #Arguments are in the order red, green, blue

@onevent
def button_left():
    global y_left, y_right, SPEED, training_state
    if training_state:
        long_press_detection = long_press_detection(button_left)
        print("long_press_detection", long_press_detection)
        if button_left == 0: 
            if not long_press_detection:  #positive reinforcement
                y_left  = -SPEED
                y_right = SPEED
                change_weights()
            else:                         #negative reinforcement
                y_left  = SPEED
                y_right = -SPEED
                change_weights()

@onevent
def button_right():
    global y_left, y_right, speed, training_state
    if training_state:
        long_press_detection = long_press_detection(button_right)
        if button_right == 0: 
            if not long_press_detection:  #positive reinforcement
                y_left  = SPEED
                y_right = -SPEED
                change_weights()
            else:                         #negative reinforcement
                y_left  = -SPEED
                y_right = SPEED
                change_weights()
        
@onevent
def button_forward():
    global y_left, y_right, speed, training_state
    if training_state:
        long_press_detection = long_press_detection(button_forward)
        if button_forward == 0: 
            if long_press_detection == 0: #positive reinforcement
                y_left  = SPEED 
                y_right = SPEED
                change_weights()
            else:                         #negative reinforcement
                y_left  = -SPEED
                y_right = -SPEED
                change_weights()
            
@onevent
def button_backward():
    global y_left, y_right, SPEED, training_state
    if training_state:
        long_press_detection = long_press_detection(button_backward)
        if button_backward == 0: 
            if not long_press_detection: 
                y_left  = -SPEED
                y_right = -SPEED
                change_weights()
            else: 
                y_left  = SPEED
                y_right = SPEED
                change_weights()
            
@onevent
def timer0():
    global prox_horizontal, leds_circle, motor_left_target, motor_right_target, SENSOR_SCALE, x, y, w_l, w_r, MOTOR_SCALE, training_state, action_performed, feedback_received, timer_c, moving
    action_performed = False
    
    if training_state and (not action_performed or not feedback_received): 
        if not action_performed and not moving: 
            # Get sensor, act, and wait for feedback
            action_performed = perceive_and_act(SENSOR_SCALE, MOTOR_SCALE)
            timer_c = timer_c + 1
            if timer_c > 5: # after 5 secs we stop until we receive a feedback
                motor_left_target = 0
                motor_right_target = 0
                moving = True
                timer_c = 0      

        if feedback_received:
            feedback_received = False
            moving = False

    if not training_state: 
        perceive_and_act(SENSOR_SCALE, MOTOR_SCALE)
        moving = False 
        feedback_received = False
        nf_leds_circle(0,0,0,0,0,0,0,0)
        action_performed = False 
    
@onevent
def timer1():
    global double_tap_detected, tap_counter, counter 
    double_tap_detected = False
    tap_counter = 0 
    counter = counter + 1 


In [91]:
await tdmclient.notebook.stop()
